In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

DATA_DIR = "/data/ml-latest"

In [2]:
!ls {DATA_DIR}

README.txt	   genome-tags.csv  movies.csv	 tags.csv     tf_idf.parquet
genome-scores.csv  links.csv	    ratings.csv  tf_idf.json


In [3]:
# помимо файла с оценками у нас есть файл с названиями и жанрами фильмов
!head {DATA_DIR}/movies.csv

In [4]:
# создаём сессию Spark
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .config("spark.driver.memory", "4g")
    .master("local[*]")
    .getOrCreate()
)

In [5]:
# файл с оценками нам всё равно понадобится
import os
import pyspark.sql.functions as sql_func

ratings = (
    spark
    .read
    .csv(
        os.path.join(DATA_DIR, "ratings.csv"),
        header=True,
        inferSchema=True
    )
    .sample(withReplacement=False, fraction=1.0, seed=0)
    .withColumn("rating_datetime", sql_func.from_unixtime("timestamp"))
    .drop("timestamp")
    .cache()
)

In [6]:
# файл с фильмами небольшой, так что его можно читать полностью
# даже если памяти доступно немного
movie_genres = (
    spark
    .read
    .csv(
        os.path.join(DATA_DIR, "movies.csv"),
        header=True,
        inferSchema=True
    )
    # парсим информацию о жанрах
    .withColumn("genres_array", sql_func.split("genres", '\|'))
    .select("movieId", sql_func.explode("genres_array").alias("genre"))
    .cache()
)

In [7]:
# получили соответствие жанров фильмам: много жанров - один фильм
movie_genres.toPandas()

,movieId,genre
0,1,Adventure
1,1,Animation
2,1,Children
3,1,Comedy
4,1,Fantasy
5,2,Adventure
6,2,Children
7,2,Fantasy
8,3,Comedy
9,3,Romance


In [8]:
# у нас есть фильмы без оценок
print("фильмов с жанрами:",
      movie_genres.select("movieId").distinct().count())
print("фильмов с оценками:", ratings.select("movieId").distinct().count())

фильмов с жанрами: 45843
фильмов с оценками: 45115


In [9]:
# создадим "профиль пользователя" (жанровые предпочтения):
# набор средних оценок фильмов одного жанра
user_profiles = (
    ratings
    .join(movie_genres, "movieId")
    .groupBy("userId", "genre")
    .agg(sql_func.avg("rating").alias("genre_rating"))
    .cache()
)

In [10]:
# посмотрим, как выглядит профиль одного из пользователей
(
    user_profiles
    .where("userId == 23")
    .orderBy(sql_func.desc("genre_rating"))
    .show()
)

+------+---------+------------------+
|userId|    genre|      genre_rating|
+------+---------+------------------+
|    23|    Drama| 4.166666666666667|
|    23|      War|               4.0|
|    23|Film-Noir|               4.0|
|    23|   Comedy|               4.0|
|    23|  Romance|               4.0|
|    23|    Crime|               4.0|
|    23|  Fantasy|3.6666666666666665|
|    23|   Action|               3.6|
|    23| Thriller|               3.5|
|    23|Adventure|               3.5|
|    23|Animation|               3.0|
|    23| Children|               3.0|
|    23|   Sci-Fi|               3.0|
+------+---------+------------------+



In [11]:
import numpy as np

# предсказываем оценку фильма как среднее по средним оценкам жанров данного пользователя
predictions = (
    ratings
    .join(movie_genres, "movieId", "left")
    .join(user_profiles, ["userId", "genre"], "left")
    .groupBy("userId", "movieId", "rating")
    .agg(sql_func.avg("genre_rating").alias("prediction"))
)
RMSE = np.sqrt(
    predictions
    .select(
        sql_func.pow(predictions.prediction - predictions.rating, 2)
        .alias("squared_error")
    )
    .agg(sql_func.avg("squared_error"))
    .first()[0]
)

In [12]:
# мы получили точность хуже, чем для линейной модели на средних весах
# но лучше, чем просто на модели со средними весами
print("точность:", RMSE)

точность: 0.8875905217636123
